# LOG-03: Network Optimization & Monte Carlo — Answer Key

In [ ]:
import pandas as pd, numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, PULP_CBC_CMD

fac = pd.read_csv("../data/facilities.csv")
cus = pd.read_csv("../data/customers.csv")
cost = pd.read_csv("../data/ship_costs.csv")

# Decision: open facility (y_f) and ship units x_{f,c}
model = LpProblem("FacilityLocation", LpMinimize)
y = {f: LpVariable(f"y_{f}", lowBound=0, upBound=1, cat="Binary") for f in fac["facility"]}
x = {(r.facility, r.customer): LpVariable(f"x_{r.facility}_{r.customer}", lowBound=0) 
     for _, r in cost.iterrows()}

# Objective: fixed + variable shipping
model += lpSum([fac.set_index("facility").loc[f,"fixed_cost"] * y[f] for f in fac["facility"]]) +                  lpSum([cost.set_index(["facility","customer"]).loc[(f,c),"unit_cost"] * x[(f,c)] 
                for f in fac["facility"] for c in cus["customer"]])
# Demand constraints
for c in cus["customer"]:
    model += lpSum([x[(f,c)] for f in fac["facility"]]) >= cus.set_index("customer").loc[c,"demand"]
# Capacity via open facilities (simple large M capacity= sum demand)
M = cus["demand"].sum()
for f in fac["facility"]:
    model += lpSum([x[(f,c)] for c in cus["customer"]]) <= M * y[f]

model.solve(PULP_CBC_CMD(msg=False))
open_fac = [f for f in fac["facility"] if y[f].value() > 0.5]
print("Open facilities:", open_fac)

# Monte Carlo: perturb demands and re-evaluate shipping cost with chosen facilities
rng = np.random.default_rng(1)
base_dem = cus.set_index("customer")["demand"]
total_costs = []
for _ in range(50):
    pert = (base_dem * rng.uniform(0.9, 1.1, len(base_dem))).round().astype(int)
    # Simple greedy assignment to open facs
    tc = 0.0
    for c, dem in pert.items():
        # pick cheapest open facility
        best = min(open_fac, key=lambda f: cost[(cost.facility==f)&(cost.customer==c)]["unit_cost"].values[0])
        u = cost[(cost.facility==best)&(cost.customer==c)]["unit_cost"].values[0]
        tc += dem * u
    total_costs.append(tc + sum(fac[fac.facility.isin(open_fac)]["fixed_cost"]))
print("Mean total cost under uncertainty:", round(np.mean(total_costs),2))
